In [1]:
import time
import numpy as np
import pandas as pd
from datetime import datetime

from datetime import datetime
import fxcmpy
from sklearn import svm
import time

In [2]:
con = fxcmpy.fxcmpy(config_file='fxcm.cfg')

In [3]:
fxcmpy.__version__

'1.2.6'

In [4]:
to_show = ['tradeId', 'amountK','currency','grossPL','isBuy']

In [5]:
lags = 3

In [6]:
model = svm.SVC(C=100, probability=True)

In [7]:
def generate_features(df, lags):
    df['Returns'] = np.log(df['Mid'] /df['Mid'].shift(1))
    print(df)
    cols = []
    for lag in range(1, lags +1):
        col = 'lag_%s' % lag
        print("col = ", col)
        df[col] = np.sign(df['Returns'].shift(lag))
        cols.append(col)
    df.dropna(inplace=True)
    return df, cols

In [8]:
candles = con.get_candles('US30', period='m1', number=100)

In [9]:
data = pd.DataFrame(candles[['askclose', 'bidclose']].mean(axis=1),
                   columns=['Mid'])

In [10]:
data, cols = generate_features(data, lags)
data.tail()

                          Mid   Returns
date                                   
2020-07-23 09:20:00  27099.55       NaN
2020-07-23 09:21:00  27093.55 -0.000221
2020-07-23 09:22:00  27091.55 -0.000074
2020-07-23 09:23:00  27092.05  0.000018
2020-07-23 09:24:00  27097.05  0.000185
2020-07-23 09:25:00  27100.05  0.000111
2020-07-23 09:26:00  27103.05  0.000111
2020-07-23 09:27:00  27104.05  0.000037
2020-07-23 09:28:00  27103.55 -0.000018
2020-07-23 09:29:00  27105.55  0.000074
2020-07-23 09:30:00  27106.05  0.000018
2020-07-23 09:31:00  27106.55  0.000018
2020-07-23 09:32:00  27108.55  0.000074
2020-07-23 09:33:00  27113.55  0.000184
2020-07-23 09:34:00  27112.55 -0.000037
2020-07-23 09:35:00  27116.05  0.000129
2020-07-23 09:36:00  27106.55 -0.000350
2020-07-23 09:37:00  27110.55  0.000148
2020-07-23 09:38:00  27118.05  0.000277
2020-07-23 09:39:00  27108.55 -0.000350
2020-07-23 09:40:00  27108.55  0.000000
2020-07-23 09:41:00  27109.55  0.000037
2020-07-23 09:42:00  27107.55 -0.000074


,Mid,Returns,lag_1,lag_2,lag_3
date,,,,,
2020-07-23 10:55:00,27094.05,-0.000055,-1.0,1.0,1.0
2020-07-23 10:56:00,27090.55,-0.000129,-1.0,-1.0,1.0
2020-07-23 10:57:00,27087.05,-0.000129,-1.0,-1.0,-1.0
2020-07-23 10:58:00,27091.05,0.000148,-1.0,-1.0,-1.0
2020-07-23 10:59:00,27093.55,0.000092,1.0,-1.0,-1.0


In [11]:
model.fit(data[cols], np.sign(data['Returns']))

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
ticks=0
position=0
tick_data=pd.DataFrame()
tick_resam=pd.DataFrame()

In [13]:
def automated_trading(data,df):
    global lags,model,ticks
    global tick_data,tick_resam,to_show
    global position
    ticks += 1
    t =datetime.now()
    if ticks % 5==0 :
        print('%3d|%s| %7.5f | %7.5f' % (ticks, str(t.time()),
                                         data['Rates'][0],data['Rates'][1]))
    
    #collecting tick data    
    tick_data=tick_data.append(pd.DataFrame(
        {'Bid':data['Rates'][0],'Ask':data['Rates'][1],
         'High':data['Rates'][2],'Low':data['Rates'][3]},
         index=[t]))  
    
    #resampling tick data
    tick_resam=tick_data[['Bid','Ask']].resample('60s',label='right').last().ffill()
    tick_resam['Mid'] = tick_resam.mean(axis=1)
    time.sleep(1)
    if len(tick_resam) > lags + 2:
        #generate a signal
        tick_resam, cols = generate_features(tick_resam, lags)
        tick_resam['Prediction'] = model.predict(tick_resam[cols])
        #entering a long position 
        if tick_resam['Prediction'].iloc[-2] >= 0 and position == 0:
            print('going long (for the first time)')
            position = 1
            order = con.create_market_buy_order('US30', 25)
            trade = True 
        elif tick_resam['Prediction'].iloc[-2] >= 0 and position == -1:
            print('going long')
            position = 1
            order = con.create_market_buy_order('US30', 50)
            trade = True
        #entering a short positon 
        elif tick_resam['Prediction'].iloc[-2] <= 0 and position == 0:
            print('going short (for the first time)')
            position = -1
            order = con.create_market_buy_order('US30', 25)
            trade = True
        elif tick_resam['Prediction'].iloc[-2] <= 0 and position == 1:
            print('going short')
            position = -1
            order = con.create_market_buy_order('US30', 50)
            trade = True

In [14]:
con.subscribe_market_data('US30', (automated_trading,))

  5|23:00:45.030037| 27086.28900 | 27087.78900
 10|23:00:55.595529| 27088.78900 | 27090.28900
 15|23:00:59.609015| 27089.28900 | 27090.78900
 20|23:01:06.671458| 27086.28900 | 27087.78900
 25|23:01:11.281405| 27089.78900 | 27091.28900
 30|23:01:15.272569| 27089.28900 | 27090.78900
 35|23:01:22.339285| 27091.78900 | 27093.28900
 40|23:01:31.598005| 27091.78900 | 27093.28900
 45|23:01:43.535213| 27093.28900 | 27094.78900
 50|23:01:57.052214| 27093.28900 | 27094.78900
 55|23:02:01.582526| 27093.78900 | 27095.28900
 60|23:02:11.052051| 27093.78900 | 27095.28900
 65|23:02:17.547997| 27094.78900 | 27096.28900
 70|23:02:22.243024| 27096.28900 | 27097.78900
 75|23:02:27.039776| 27095.28900 | 27096.78900
 80|23:02:32.071954| 27095.78900 | 27097.28900
 85|23:02:35.599463| 27092.78900 | 27094.28900
 90|23:02:41.286837| 27095.78900 | 27097.28900
 95|23:02:45.572968| 27093.78900 | 27095.28900
100|23:02:52.665867| 27093.78900 | 27095.28900
105|23:02:56.649532| 27094.28900 | 27095.78900
110|23:03:03.

Exception in thread Thread-198:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jeffreywu/.envs/fxcm1/lib/python3.6/site-packages/socketio/client.py", line 592, in _handle_eio_message
    self._handle_event(pkt.namespace, pkt.id, pkt.data)
  File "/Users/jeffreywu/.envs/fxcm1/lib/python3.6/site-packages/socketio/client.py", line 476, in _handle_event
    r = self._trigger_event(data[0], namespace, *data[1:])
  File "/Users/jeffreywu/.envs/fxcm1/lib/python3.6/site-packages/socketio/client.py", line 526, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "/Users/jeffreywu/.envs/fxcm1/lib/python3.6/site-packages/fxcmpy/fxcmpy.py", line 2555,

                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00  27097.789  27099.289  27098.539  0.000018
2020-07-23 23:06:00  27096.789  27098.289  27097.539 -0.000037
col =  lag_1
col =  lag_2
col =  lag_3
195|23:05:12.526107| 27096.78900 | 27098.28900
                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00  27097.789  27099.289  27098.539  0.000018
2020-07-23 23:06:00  27096.789  27098.289  27097.539 -0.000037
col =  lag_1
col =  lag_2
col = 

                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00  27097.789  27099.289  27098.539  0.000018
2020-07-23 23:06:00  27097.289  27098.789  27098.039 -0.000018
2020-07-23 23:07:00  27099.789  27101.289  27100.539  0.000092
col =  lag_1
col =  lag_2
col =  lag_3
                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00  27097.789  27099.289  27098.539  0.000018
2020-07-23 23:06:00  27097.289  27098.789  27098.039 -0.000018
2020-07-23 23:07

                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00  27097.789  27099.289  27098.539  0.000018
2020-07-23 23:06:00  27097.289  27098.789  27098.039 -0.000018
2020-07-23 23:07:00  27101.789  27103.289  27102.539  0.000166
2020-07-23 23:08:00  27101.289  27102.789  27102.039 -0.000018
col =  lag_1
col =  lag_2
col =  lag_3
going short
240|23:07:01.524124| 27101.28900 | 27102.78900
                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00 

                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00  27097.789  27099.289  27098.539  0.000018
2020-07-23 23:06:00  27097.289  27098.789  27098.039 -0.000018
2020-07-23 23:07:00  27101.789  27103.289  27102.539  0.000166
2020-07-23 23:08:00  27103.789  27105.289  27104.539  0.000074
col =  lag_1
col =  lag_2
col =  lag_3
                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00  27097.789  27099.289  27098.539  0.000018
2020-07-23 23:06

                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00  27097.789  27099.289  27098.539  0.000018
2020-07-23 23:06:00  27097.289  27098.789  27098.039 -0.000018
2020-07-23 23:07:00  27101.789  27103.289  27102.539  0.000166
2020-07-23 23:08:00  27106.789  27108.289  27107.539  0.000184
2020-07-23 23:09:00  27108.789  27110.289  27109.539  0.000074
col =  lag_1
col =  lag_2
col =  lag_3
going long
                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-

In [15]:
con.unsubscribe_market_data('US30')

295|23:08:24.058418| 27105.78900 | 27107.28900
                           Bid        Ask        Mid   Returns
2020-07-23 23:01:00  27089.289  27090.789  27090.039       NaN
2020-07-23 23:02:00  27093.289  27094.789  27094.039  0.000148
2020-07-23 23:03:00  27093.289  27094.789  27094.039  0.000000
2020-07-23 23:04:00  27097.289  27098.789  27098.039  0.000148
2020-07-23 23:05:00  27097.789  27099.289  27098.539  0.000018
2020-07-23 23:06:00  27097.289  27098.789  27098.039 -0.000018
2020-07-23 23:07:00  27101.789  27103.289  27102.539  0.000166
2020-07-23 23:08:00  27106.789  27108.289  27107.539  0.000184
2020-07-23 23:09:00  27105.789  27107.289  27106.539 -0.000037
col =  lag_1
col =  lag_2
col =  lag_3


In [16]:
tick_data

,Ask,Bid,High,Low
2020-07-23 23:00:42.560313,27087.789,27086.289,27170.6,26930.1
2020-07-23 23:00:43.547104,27087.789,27086.289,27170.6,26930.1
2020-07-23 23:00:44.041551,27087.789,27086.289,27170.6,26930.1
2020-07-23 23:00:44.542645,27088.289,27086.789,27170.6,26930.1
2020-07-23 23:00:45.030037,27087.789,27086.289,27170.6,26930.1
2020-07-23 23:00:46.702940,27087.789,27086.289,27170.6,26930.1
2020-07-23 23:00:47.054727,27087.289,27085.789,27170.6,26930.1
2020-07-23 23:00:47.627209,27087.789,27086.289,27170.6,26930.1
2020-07-23 23:00:51.045117,27087.789,27086.289,27170.6,26930.1
2020-07-23 23:00:55.595529,27090.289,27088.789,27170.6,26930.1
